In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import matplotlib.gridspec as gridspec
import ast
import sys
sys.path.append('machine-scientist/')
sys.path.append('machine-scientist/Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.stats import bootstrap

In [5]:
#Read error dataframe for all simulations
function='tanh'
errors=pd.read_csv('../data/'+ 'errors_' + str(function) + '.csv',index_col=0)
sigma0=errors[errors['sigma']==0]
display(sigma0)

,sigma,mae_nn_train,mae_nn_test,mae_mdl_train,mae_mdl_test,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test,n,r
0,0.0,0.038346,0.031069,0.001119,9.678334e-03,0.048355,0.034498,0.001325,1.169205e-02,0,0
1,0.0,0.011996,0.106369,0.002785,3.321931e-01,0.015287,0.112211,0.003340,4.975634e-01,1,0
2,0.0,0.045330,0.568604,0.001012,5.818908e-02,0.119820,0.574555,0.001299,7.225381e-02,2,0
3,0.0,0.017690,0.168191,0.000309,4.174655e-02,0.043636,0.169878,0.000399,4.874519e-02,3,0
4,0.0,0.015367,0.066641,0.000170,2.948595e-03,0.019958,0.070403,0.000248,4.091270e-03,4,0
5,0.0,0.007376,0.071449,0.000487,1.408517e-01,0.012901,0.088791,0.000684,1.689972e-01,5,0
6,0.0,0.006577,0.091553,0.000324,1.811364e-02,0.016011,0.093125,0.000389,2.311304e-02,6,0
7,0.0,0.007777,0.060940,0.000700,2.582044e-02,0.014318,0.061244,0.000904,3.364340e-02,7,0
8,0.0,0.007191,0.098977,0.000051,5.952950e-02,0.015660,0.099978,0.000071,8.638921e-02,8,0
9,0.0,0.002033,0.026621,0.000014,2.454331e-01,0.002766,0.032740,0.000016,6.222422e-01,9,0


In [7]:
columns_errors=['rmse_nn_train','rmse_nn_test','rmse_mdl_train','rmse_mdl_test']
medians=sigma0.groupby(['sigma'],as_index=False)[columns_errors].median()
display(medians)

means=sigma0.groupby(['sigma'],as_index=False)[columns_errors].mean()
display(means)

,sigma,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test
0,0.0,0.017356,0.08885,0.000583,0.046586


,sigma,rmse_nn_train,rmse_nn_test,rmse_mdl_train,rmse_mdl_test
0,0.0,0.031831,0.137274,0.024321,inf
